## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia

In [108]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd
import datetime

## TASK 1: Request the Falcon9 Launch Wiki page from its URL

In [2]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Send an HTTP GET request to the webpage
response =  requests.get(static_url)
response.status_code

200

In [3]:
# Store the HTML content in a variable
html_content = response.text

In [4]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
soup = BeautifulSoup(html_content, 'html.parser')
soup.title

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

## TASK 2: Extract all column/variable names from the HTML table header

Let's try to find all tables on the wiki page first.

In [5]:
html_tables = soup.find_all('table')
html_tables.__len__() #hay 25 tablas

25

Starting from the third table is our target table contains the actual launch records.

In [6]:
first_launch_table = html_tables[2]

In [7]:
headers = first_launch_table.find_all('th')
headers[0:5]

[<th scope="col">Flight No.
 </th>,
 <th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
 </th>,
 <th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11"><span class="cite-bracket">[</span>b<span class="cite-bracket">]</span></a></sup>
 </th>,
 <th scope="col">Launch site
 </th>,
 <th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12"><span class="cite-bracket">[</span>c<span class="cite-bracket">]</span></a></sup>
 </th>]

In [45]:
# trying code ways (just practice)
cols = []
for i in headers:
    cols.append(i.text.strip())
cols

['Flight No.',
 'Date andtime ()',
 '',
 'Launch site',
 'Payload',
 'Payload mass',
 'Orbit',
 'Customer',
 'Launchoutcome',
 '',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7']

In [8]:
def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br): #br is 'line break' like \n
        row.br.extract()
    if row.a: # a is anchor for links
        row.a.extract()
    if row.sup: #used to define superscript text. Superscript text appears slightly above the normal line and is often rendered in a smaller font
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name  

In [28]:
column_names = []
for th in headers:
    col = extract_column_from_header(th)
    
    if col != None and len(col)>0:
        column_names.append(col)

In [29]:
column_names

['Flight No.',
 'Date and time ( )',
 'Launch site',
 'Payload',
 'Payload mass',
 'Orbit',
 'Customer',
 'Launch outcome']

## TASK 3: Create a data frame by parsing the launch HTML tables

We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe

In [11]:
launch_dict= dict.fromkeys(column_names)
# Remove an irrelvant column
del launch_dict['Date and time ( )']
launch_dict

{'Flight No.': None,
 'Launch site': None,
 'Payload': None,
 'Payload mass': None,
 'Orbit': None,
 'Customer': None,
 'Launch outcome': None}

In [257]:
# Let's initializa the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster'] = []
launch_dict['Booster landing'] = []
launch_dict['Date'] = []
launch_dict['Time'] = []

In [258]:
launch_dict

{'Flight No.': [],
 'Launch site': [],
 'Payload': [],
 'Payload mass': [],
 'Orbit': [],
 'Customer': [],
 'Launch outcome': [],
 'Version Booster': [],
 'Booster landing': [],
 'Date': [],
 'Time': []}

Next, we just need to fill up the launch_dict with launch records extracted from table rows\
Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links B0004.1[8], missing values N/A [e], inconsistent formatting, etc.

To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the launch_dict. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:

In [ ]:
table1 = html_tables[2]
table1_headers = table1.find_all('th')
table1_column_names = []

for th in table1_headers:
    name = extract_column_from_header(th)

    if col != None and len(col)>0:
        table1_column_names.append(col)

print(table1_column_names)

table1_rows = table1.find_all('tr')

for row in table1_rows:
    print(row)

In [171]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass

In [259]:
extracted_row = 0

#Extract each table
for table_number, table in enumerate(soup.find_all('table')):
    #get table row
    for rows in table.find_all('tr'):
        #check to see if first table heading is as number corresponding to launch a number
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip() # strip quita los espacios del final de cada string(\n)
                flag = flight_number.isdigit() #true or false for numbers
        else:
            flag = False

        row = rows.find_all('td')
        # FLIGHT NUMBER
        if flag:   #if it is TRUE is a numner, so save data in the dictonary
            extracted_row +=1
            launch_dict['Flight No.'].append(flight_number)
            
            ''' esto hace la funcion del lab:
            for data_time in list(row[0].strings):
            data = data_time
            data = datatimelist[0].strip(',')
            time2= datatimelist[1]
            print(time2)
            '''
            # DATE AND TIME
            datatimelist = row[0].text.split(',') # ['4 June 2010', '18:45\n']
            date_list = datatimelist[0:1] # ['4 June 2010']
            date = date_list[0]  # 4 June 2010
            launch_dict['Date'].append(date) # add value to its key dictionary

            time_list = datatimelist[1:2] # ['18:45\n']
            if len(time_list) == 0:
                time = '00:00'
            else:
                time = time_list[0][0:5] #los primeros 5 characteres 18:45 
            launch_dict['Time'].append(time) # add value to its key dictionary

            # VERSION BOOSTER
            bv = booster_version(row[1])
            if not(bv):
                bv = row[1].a.string #jala el string que hay dentro de a(anchor/link)
            launch_dict['Version Booster'].append(bv)

            # Launch Site
            launch_site = row[2].a.string
            launch_dict['Launch site'].append(launch_site)
  
            # Payload
            payload = row[3].a.string
            launch_dict['Payload'].append(payload)

            # Payload Mass
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)

            # Orbit
            orbit = row[5].a.string
            launch_dict['Orbit'].append(orbit)
            
            # Customer
            if row[6].string == None:
                customer = row[6].a.string
            elif row[6].string:
                customer = row[6].string.strip()
            launch_dict['Customer'].append(customer)
             
            # Launch outcome
            launch_outcome = list(row[7].strings)[0].strip()
            launch_dict['Launch outcome'].append(launch_outcome)

            # Booster landing
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)

After you have fill in the parsed launch record values into launch_dict, you can create a dataframe from it.

In [263]:
df = pd.DataFrame({key:pd.Series(value) for key, value in launch_dict.items()}) # items devuelve la key y values en una tupla

In [264]:
df

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
0,1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,F9 v1.07B0003.18,Failure,4 June 2010,18:45
1,2,CCAFS,Dragon,0,LEO,NASA,Success,F9 v1.07B0004.18,Failure,8 December 2010,15:43
2,3,CCAFS,Dragon,525 kg,LEO,NASA,Success,F9 v1.07B0005.18,No attempt\n,22 May 2012,07:44
3,4,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success,F9 v1.07B0006.18,No attempt,8 October 2012,00:35
4,5,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success,F9 v1.07B0007.18,No attempt\n,1 March 2013,15:10
...,...,...,...,...,...,...,...,...,...,...,...
116,117,CCSFS,Starlink,"15,600 kg",LEO,SpaceX,Success,F9 B5B1051.10657,Success,9 May 202106:42[656]\n,00:00
117,118,KSC,Starlink,"~14,000 kg",LEO,SpaceX,Success,F9 B5B1058.8660,Success,15 May 202122:56[659]\n,00:00
118,119,CCSFS,Starlink,"15,600 kg",LEO,SpaceX,Success,F9 B5B1063.2665,Success,26 May 202118:59[664]\n,00:00
119,120,KSC,SpaceX CRS-22,"3,328 kg",LEO,NASA,Success,F9 B5B1067.1668,Success,3 June 202117:29[667]\n,00:00


In [265]:
df.to_csv('spacex_web_scraped.csv', index=False)